In [14]:
import pandas as pd
import numpy as np
import torch

In [5]:
results_df = pd.read_csv("Saved/preds_targets_probs_40e_10f.csv")
results_df

,Fold,ImagePaths,Predictions,Targets
0,0,images/3cca7146-1010_463.jpg,[4.2817576e-04 3.7263100e-05 1.7458877e-04 1.7...,[0 0 0 0]
1,0,images/f3fbe6f6-1129_73.jpg,[0.99998367 0.99999046 0.9998697 0.99998176],[1 1 1 1]
2,0,images/e5a1fc17-0828_136.jpg,[9.5956811e-06 7.4980258e-06 9.9998426e-01 2.3...,[0 0 1 0]
3,0,images/6a310bf6-1001_209.jpg,[1.6568579e-04 6.3139241e-04 3.0443003e-05 9.9...,[0 0 0 1]
4,0,images/a8b62ddb-1010_843.jpg,[3.0744355e-05 5.4354418e-06 5.0321165e-07 2.4...,[0 0 0 0]
...,...,...,...,...
5500,9,images/a6d77a30-0828_753.jpg,[2.8116713e-04 9.9957854e-01 9.9933845e-01 9.9...,[0 1 1 1]
5501,9,images/50f4bcfb-1022_62.jpg,[0.9999949 0.99998116 0.9998857 0.9999963 ],[1 1 1 1]
5502,9,images/3e7a2915-1010_738.jpg,[0.9713037 0.99998343 0.99992 0.00730028],[1 1 1 0]
5503,9,images/16c773b0-1001_394.jpg,[0.01815899 0.0004658 0.00026154 0.00175294],[1 0 0 1]


In [32]:
# parse string arrays
def parse_array(s):
    return np.fromstring(s.strip("[]"), sep=" ")

preds = results_df["Predictions"].apply(parse_array).to_list()
targets = results_df["Targets"].apply(parse_array).to_list()

# Convert to numpy arrays
preds = np.stack(preds)
targets = np.stack(targets).astype(int)

In [31]:
subset_acc = MultilabelExactMatch(num_labels = 4)
subset_acc(preds, targets)

tensor(0.9275)

In [41]:
def compute_metrics(preds, targets, target_names):

    preds = torch.tensor(preds, dtype=torch.float32)
    targets = torch.tensor(targets, dtype=torch.long)

    tar_np = targets.detach().cpu().numpy()
    pred_np = preds.detach().cpu().numpy()
    preds_binary = (pred_np >= 0.5).astype(int) 

    subset_acc = MultilabelExactMatch(num_labels = 4)
    macro_acc = MultilabelAccuracy(num_labels = 4, average = "macro")
    micro_acc = MultilabelAccuracy(num_labels = 4, average = "micro")
    weight_acc = MultilabelAccuracy(num_labels = 4, average = "weighted")
    per_label_acc = MultilabelAccuracy(num_labels = 4, average = None)
    
    ham_macro = MultilabelHammingDistance(num_labels = 4 , average='macro')
    ham_micro = MultilabelHammingDistance(num_labels = 4 , average='micro')
    multilabel_auroc_ma = MultilabelAUROC(num_labels = 4, average = "macro", thresholds = None)    # multiclass AUROC
    multilabel_auprc_ma = MultilabelAveragePrecision(num_labels = 4, average = "macro", thresholds = None)    # multiclass AUPRC 
    multilabel_auroc_mi = MultilabelAUROC(num_labels = 4, average = "micro", thresholds = None)    # multiclass AUROC
    multilabel_auprc_mi = MultilabelAveragePrecision(num_labels = 4, average = "micro", thresholds = None)    # multiclass AUPRC 

    print(f"Macro Acc: {macro_acc(preds, targets):.4f}")
    print(f"Per Label Acc: {per_label_acc(preds, targets).detach().cpu().numpy()}")
    print(f"Subset Acc: {subset_acc(preds, targets):.4f} | Marco-Ham: {ham_macro(preds, targets):.4f}\n")
    
    print(f"Macro AUROC:  {multilabel_auroc_ma(preds, targets):.4f}         Macro AUPRC:  {multilabel_auprc_ma(preds, targets):.4f}")

target_names = ["Right Upper", "Left Upper", "Left Lower", "Right Lower"]
compute_metrics(preds, targets, target_names)

Average Performance across all folds
Total Samples: 5505

TorchMetrics
Macro Acc: 0.9781 | Micro Acc: 0.9781 | Weighted Acc: 0.9781
Per Label Acc: [0.97947323 0.9782016  0.9782016  0.97656673]
Subset Acc: 0.9275 | Marco-Ham: 0.0219 | Micro-Ham: 0.0219

Macro AUROC:  0.9908         Macro AUPRC:  0.9873
Micro AUROC:  0.9909         Micro AUPRC:  0.9877

              precision    recall  f1-score   support

 Right Upper     0.9794    0.9692    0.9743      2211
  Left Upper     0.9775    0.9688    0.9732      2247
  Left Lower     0.9744    0.9760    0.9752      2414
 Right Lower     0.9743    0.9714    0.9728      2378

   micro avg     0.9763    0.9715    0.9739      9250
   macro avg     0.9764    0.9714    0.9739      9250
weighted avg     0.9763    0.9715    0.9739      9250
 samples avg     0.9782    0.9742    0.9629      9250



In [47]:
import numpy as np
import pandas as pd
import torch
from torchmetrics.classification import (
    MultilabelAccuracy,
    MultilabelHammingDistance,
    MultilabelAUROC,
    MultilabelAveragePrecision,
    MultilabelExactMatch,
)

CSV_PATH = "Saved/preds_targets_probs_40e_10f.csv"  # change if needed
FOLD_COL = "Fold"
PRED_COL = "Predictions"
TARG_COL = "Targets"
THRESHOLD = 0.5

# If you have explicit names, set them here. Otherwise they will be auto-generated.
TARGET_NAMES = ["Right Upper", "Left Upper", "Left Lower", "Right Lower"]

# ----------------------------
# Utils
# ----------------------------
def parse_array(s: str) -> np.ndarray:
    return np.fromstring(str(s).strip("[]"), sep=" ")

def _compute_fold_metrics(preds_np: np.ndarray,
                          targets_np: np.ndarray,
                          threshold: float = 0.5) -> dict:

    preds_t   = torch.tensor(preds_np, dtype=torch.float32)
    targets_t = torch.tensor(targets_np, dtype=torch.long)
    num_labels = preds_t.size(1)

    subset_acc = MultilabelExactMatch(num_labels=num_labels)
    macro_acc  = MultilabelAccuracy(num_labels=num_labels, average="macro",    threshold=threshold)
    per_label  = MultilabelAccuracy(num_labels=num_labels, average=None,       threshold=threshold)

    auroc_ma   = MultilabelAUROC(num_labels=num_labels, average="macro")
    auprc_ma   = MultilabelAveragePrecision(num_labels=num_labels, average="macro")

    out = {
        "n":             int(preds_t.size(0)),
        "subset_acc":    float(subset_acc(preds_t, targets_t)),
        "macro_acc":     float(macro_acc(preds_t, targets_t)),
        "macro_auroc":   float(auroc_ma(preds_t, targets_t)),
        "macro_auprc":   float(auprc_ma(preds_t, targets_t)),
        "per_label_acc": per_label(preds_t, targets_t).detach().cpu().numpy(),  # shape (C,)
    }
    return out

def compute_metrics_by_fold(results_df: pd.DataFrame,
                            target_names=None,
                            fold_col: str = "Fold",
                            pred_col: str = "Predictions",
                            targ_col: str = "Targets",
                            threshold: float = 0.5):

    first_row = results_df[pred_col].iloc[0]
    C = parse_array(first_row).shape[0]
    if target_names is None:
        target_names = [f"Label_{i}" for i in range(C)]
    else:
        assert len(target_names) == C, f"len(target_names)={len(target_names)} but C={C}"

    folds = sorted(results_df[fold_col].unique())
    per_fold_rows, per_label_rows = [], []

    for f in folds:
        df_f = results_df[results_df[fold_col] == f]

        preds_np   = np.stack(df_f[pred_col].apply(parse_array).to_list()).astype(np.float32)  # (N_f, C)
        targets_np = np.stack(df_f[targ_col].apply(parse_array).to_list()).astype(np.int64)    # (N_f, C)

        fm = _compute_fold_metrics(preds_np, targets_np, threshold=threshold)

        row = {"fold": f, "n": fm["n"]}
        for k in ["subset_acc","macro_acc",
                 "macro_auroc","macro_auprc"]:
            row[k] = fm[k]
        per_fold_rows.append(row)

        # per-label accuracy for this fold
        pla = fm["per_label_acc"]
        pla_row = {"fold": f, **{f"acc_{name}": float(pla[i]) for i, name in enumerate(target_names)}}
        per_label_rows.append(pla_row)

    per_fold_df = pd.DataFrame(per_fold_rows).set_index("fold").sort_index()
    per_label_acc_df = pd.DataFrame(per_label_rows).set_index("fold").sort_index()
    per_label_acc_df.loc["mean"] = per_label_acc_df.mean(axis=0)

    return per_fold_df, per_label_acc_df, target_names

def summarize_across_folds(per_fold_df: pd.DataFrame):

    if "n" not in per_fold_df.columns:
        raise ValueError("per_fold_df must contain 'n'")

    totals = {"n": int(per_fold_df["n"].sum())}
    metric_cols = [c for c in per_fold_df.columns if c != "n"]

    unweighted_mean = per_fold_df[metric_cols].mean().to_dict()
    std_over_folds = per_fold_df[metric_cols].std(ddof=1).to_dict()

    return unweighted_mean, totals, std_over_folds

def pooled_metrics(results_df: pd.DataFrame,
                   pred_col="Predictions",
                   targ_col="Targets",
                   threshold=0.5) -> dict:

    preds_np = np.stack(results_df[pred_col].apply(parse_array).to_list()).astype(np.float32)
    targs_np = np.stack(results_df[targ_col].apply(parse_array).to_list()).astype(np.int64)
    return _compute_fold_metrics(preds_np, targs_np, threshold=threshold)

# ----------------------------
# Run
# ----------------------------
if __name__ == "__main__":
    # Load CSV
    results_df = pd.read_csv(CSV_PATH)

    # Compute per-fold metrics
    per_fold_df, per_label_acc_df, target_names = compute_metrics_by_fold(
        results_df,
        target_names=TARGET_NAMES,
        fold_col=FOLD_COL,
        pred_col=PREDCOL if (PREDCOL := PRED_COL) else "Predictions",  # defensive alias
        targ_col=TARGCOL if (TARGCOL := TARG_COL) else "Targets",
        threshold=THRESHOLD
    )

    # Summaries across folds
    unweighted_mean, totals, std_over_folds = summarize_across_folds(per_fold_df)

    # Pooled metrics
    pooled = pooled_metrics(results_df, pred_col=PRED_COL, targ_col=TARG_COL, threshold=THRESHOLD)

    # ----------------------------
    # Print results
    # ----------------------------
    pd.set_option("display.width", 180)
    pd.set_option("display.max_columns", 200)

    print("\nPer-fold metrics:")
    print(per_fold_df.round(4))

    print("\nPer-label accuracy per fold:")
    print(per_label_acc_df.round(4))

    print(f"\nTotal samples (sum over folds): {totals['n']}")

    print("\nAverage over folds:")
    for k, v in unweighted_mean.items():
        print(f"{k}: {v:.4f}")

    print("\nStd over folds:")
    for k, v in std_over_folds.items():
        print(f"{k}: {v:.4f}")

    print("\nPooled over all folds:")
    for k, v in pooled.items():
        if k == "per_label_acc":
            continue
        if k == "n":
            print(f"{k}: {int(v)}")
        else:
            print(f"{k}: {v:.4f}")



Per-fold metrics:
        n  subset_acc  macro_acc  macro_auroc  macro_auprc
fold                                                      
0     550      0.9255     0.9786       0.9934       0.9852
1     551      0.9274     0.9778       0.9944       0.9915
2     552      0.9167     0.9733       0.9910       0.9875
3     551      0.9256     0.9787       0.9918       0.9905
4     551      0.9310     0.9791       0.9883       0.9850
5     548      0.9215     0.9767       0.9905       0.9903
6     551      0.9328     0.9773       0.9939       0.9930
7     551      0.9383     0.9819       0.9929       0.9922
8     549      0.9199     0.9763       0.9858       0.9842
9     551      0.9365     0.9814       0.9939       0.9895

Per-label accuracy per fold:
      acc_Right Upper  acc_Left Upper  acc_Left Lower  acc_Right Lower
fold                                                                  
0              0.9836          0.9818          0.9836           0.9655
1              0.9855         